In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize as sco

import hx_hydraulic as hxhy

In [ ]:
df = pd.read_csv(r'C:\Users\valen\Downloads\PL_Maddalena.csv')

In [ ]:
df.rename(columns={'x': 'Vdot', ' y': 'PL'}, inplace=True)
df.sort_values(by='Vdot', inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['Vdot_m3s'] = df['Vdot']/3.6e6
A = np.pi*0.0354**2/4
df['V'] = df['Vdot_m3s']/A

In [ ]:
bounds = ([0.],[np.inf])
def polyn2(q,b):
    return b*q**2
popt, pcov = sco.curve_fit(polyn2, df['V'], df['PL'],bounds=bounds)

q_array = np.arange(0,2,0.05)
dfe = pd.DataFrame()
dfe['V'] = q_array
dfe['PL'] = polyn2(q_array,*popt)

In [ ]:
df

In [ ]:
dfe

In [ ]:
popt

In [ ]:
# D_poss = np.arange(0.0129,0.0135,0.0001)

# for D in D_poss:
#     print(D)
#     duct = hxhy.duct(shape='tubular',D=D,L=0.130,k=1e-3)    
#     df['PL_Di='+str(round(D*1000,2))+'mm'] = df['Vdot'].apply(lambda x : duct.regular_PL(x,fluid_dict={'name':'MPG','glycol_rate':0.4}, p=1e5, T=293.15))

In [ ]:
def diff_D(D):
    duct = hxhy.duct(shape='tubular',D=D,L=0.130,k=1e-3)
    df['PL_D'] = df['Vdot'].apply(lambda x : duct.regular_PL(x,fluid_dict={'name':'MPG','glycol_rate':0.4}, p=1e5, T=293.15))
    return np.sum((df['PL']-df['PL_D'])**2)

In [ ]:
def diff_L(L):
    duct = hxhy.duct(shape='tubular',D=0.0354,L=L,k=1e-3)
    df['PL_D'] = df['Vdot'].apply(lambda x : duct.regular_PL(x,fluid_dict={'name':'MPG','glycol_rate':0.4}, p=1e5, T=293.15))
    return np.sum((df['PL']-df['PL_D'])**2)

In [ ]:
D_min = sco.minimize(diff_D,0.013,method='Nelder-Mead')['x'][0]

In [ ]:
L_min = sco.minimize(diff_L, 0.5, method='Nelder-Mead')['x'][0]

In [ ]:
L_min

In [ ]:
duct = hxhy.duct(shape='tubular',D=D,L=0.130,k=1e-3)
df['PL_D'] = df['Vdot'].apply(lambda x : duct.regular_PL(x,fluid_dict={'name':'MPG','glycol_rate':0.4}, p=1e5, T=293.15))

In [ ]:
duct = hxhy.duct(shape='tubular',D=D_min,L=0.130,k=1e-3)
df['PL_D'] = df['Vdot'].apply(lambda x : duct.regular_PL(x,fluid_dict={'name':'MPG','glycol_rate':0.4}, p=1e5, T=293.15))

duct = hxhy.duct(shape='tubular',D=0.0354,L=L_min,k=1e-3)
df['PL_L'] = df['Vdot'].apply(lambda x : duct.regular_PL(x,fluid_dict={'name':'MPG','glycol_rate':0.4}, p=1e5, T=293.15))

In [ ]:
plt.plot(df['Vdot'],df['PL'],label='Data')
plt.plot(dfe['Vdot'],dfe['PL'])

plt.xlim(0,800)
plt.ylim(0,2)

In [ ]:
popt

In [ ]:
plt.plot(df['Vdot'],df['PL'],label='Polynomial fit')
plt.plot(df['Vdot'],df['PL_D'],label='D')
plt.plot(df['Vdot'],df['PL_L'],label='L')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.xlim(0,800)
plt.ylim(0,2)

In [ ]:
Di_DN20 = 0.020
Di_DN25 = 0.026
Di_DN32 = 0.033